In [2]:
import pyaudio
import wave
import numpy as np
import math
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
message = "hello from the other side hakuna matata abcdefghijklmnopqrstuvwxyz ."

# Transmitter

## Char-Freq Table

We should compute this as fast as possible and then start transmitting to the receiver. **Figure out if there are some frequencies that we should let the more common characters take.** If so, we can create a Huffman Encoding table but that's going to involve a lot of overhead so make sure it's worth it. Start transmitting as soon as this is done building and transmit asynchronously while doing following step. 

In [4]:
t = 1
samples = 10000 # This is super weird - 333 and 3333 work perfectly but 10000 doesn't. Even 33 works well?!

# Create a set with every character in file 

# Assign each possible char an integer (frequency) in a hash set

# Send sinusoids - two for each mapping because one for char and one for frequency 



# Frequency range (will need to test this)
freq_range = range(5000, 15000, 300)

chars = "abcdefghijklmnopqrstuvwxyz ."

char_freq_table = {}
projection_matrix = []

for i in range(len(chars)):
    char_freq_table[chars[i]] = freq_range[i]
    projection_matrix.append(np.sin(2 * np.pi * freq_range[i] * np.arange(t * samples) / samples))
print(char_freq_table['a'])
print(char_freq_table['m'])

5000
8600


## Encode Chars to Frequencies

Separate .txt file into chunks. Give each char in a chunk a phase and then encode frequencies according to char-freq table. **There doesn't have to be a one-to-one correspondence between sinusoid and char.** If I can fit multiple chars into one sinusoid, that's better. Only ~50 commonly used characters and thousands of frequencies. 

In [143]:
chunk_size = 5

split_message = [message[i:i+chunk_size] for i in range(0, len(message), chunk_size)]

message_freq = []

for chunk in split_message:
    message_freq.append([char_freq_table[char] for char in chunk])

print(message_freq)

[[7100, 6200, 8300, 8300, 9200], [12800, 6500, 10100, 9200, 8600], [12800, 10700, 7100, 6200, 12800], [9200, 10700, 7100, 6200, 10100], [12800, 10400, 7400, 5900, 6200], [12800, 7100, 5000, 8000, 11000], [8900, 5000, 12800, 8600, 5000], [10700, 5000, 10700, 5000, 12800], [5000, 5300, 5600, 5900, 6200], [6500, 6800, 7100, 7400, 7700], [8000, 8300, 8600, 8900, 9200], [9500, 9800, 10100, 10400, 10700], [11000, 11300, 11600, 11900, 12200], [12500, 12800, 13100]]


## Create Superposition

In [144]:
# I think we'll need to communicate length of the message and chunk size 
sigma = .5
delay = t / chunk_size 

superpositions = []

# For each chunk, generate 3 sinusoids 
for chunk in message_freq:
    # Create 3 sinusoids 
    sins = [np.sin(2 * np.pi * freq * np.arange(t * samples) / samples) for freq in chunk]
    chunk_super = np.zeros(samples)
    # Put 3 sinusoids in superposition with delay 
    for i in range(samples):
        for j in range(len(sins)): # Use len(sins) because the group might be less than chunks
            if i >= int(j * delay * samples):
                #chunk_super[i] += sins[j][i - int(j * delay)]
                chunk_super[i] += sins[j][i] # No delay - just removing first parts of later sinusoids
    #chunk_super += np.random.normal(loc=0.0, scale=sigma, size=[samples,])
    superpositions.append(chunk_super)

print(superpositions[0])

[ 0.         -0.96858316  0.48175367 ...  0.9139903   1.05320337
  3.8874973 ]


## Test Projection

In [145]:
def projection(superpositions):
    proj_results = [] 
    for chunk in superpositions:
        largest = np.argsort(np.dot(chunk, np.transpose(projection_matrix)))[::-1][:chunk_size]
        proj_results.extend(largest) 
    return "".join([chars[i] for i in proj_results])
    
print("Decoded versoin: ")
print(projection(superpositions))

# So for some reason, 'a' maps to 'm' and 'z' maps to 'q'

Decoded versoin: 
hleoc from thefother side hkunn miwt fwibcdelfghijklmnopqrstuvwxyz .op


## Create Waveforms

In [146]:
def gen_waveforms(chunk_size, message_freq):
    # Separate message into an array of strings of length chunk_size 
    split_message = [message[i:i+chunk_size] for i in range(0, len(message), chunk_size)]
    
    # Encode each string in split_message as a superposition of frequencies 
    superpositions = [gen_sinusoids(chunk_size, [])]
    

waveforms = []
chunk_size = 1 
waveforms = gen_waveforms(chunk_size, message_freq)

NameError: name 'gen_sinusoids' is not defined

In [41]:
test = [message[i:i+3] for i in range(0, len(message), 3)]
print(test)

['hel', 'lo ', 'fro', 'm t', 'he ', 'oth', 'er ', 'sid', 'e']


## Transmit 

We have to transmit according to some ECC that allows for ambient noise and sudden noise. 

In [37]:
p = pyaudio.PyAudio()
length = .1 # number of seconds to play sound 

# Generate a wave for full message (change this to generate one chunk at a time)
wavedata = ''

for freq in message_freq:
    bitrate = 16000 # number of frames per second 
    frequency = freq
    if frequency > bitrate:
        bitrate = frequency + 100 
    num_frames = int(bitrate * length)
    rest_frames = num_frames % bitrate 
    
    #generate waves 
    for j in range(num_frames):
        wavedata += chr(int(math.sin(j / ((bitrate / frequency) / math.pi)) * 127 + 128))
    for j in range(rest_frames):
        wavedata += chr(128)

print(len(wavedata))

# Now play wavedata
stream = p.open(format=p.get_format_from_width(1), channels=1, rate=bitrate, output=True)
stream.write(wavedata)
stream.stop_stream()
stream.close()
p.terminate()

80000


# Receiver

## Start Recording Audio

[Source](https://stackoverflow.com/questions/35344649/reading-input-sound-signal-using-python)

In [2]:
# CHUNK = 1024
# FORMAT = pyaudio.paInt16
# CHANNELS = 2
# RATE = 44100
# RECORD_SECONDS = 5
# WAVE_OUTPUT_FILENAME = "output.wav"

# p = pyaudio.PyAudio()

# stream = p.open(format=FORMAT,
#                 channels=CHANNELS,
#                 rate=RATE,
#                 input=True,
#                 frames_per_buffer=CHUNK)

# print("* recording")

# frames = []

# for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
#     data = stream.read(CHUNK)
#     frames.append(data)

# print("* done recording")

# stream.stop_stream()
# stream.close()
# p.terminate()

# wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
# wf.setnchannels(CHANNELS)
# wf.setsampwidth(p.get_sample_size(FORMAT))
# wf.setframerate(RATE)
# wf.writeframes(b''.join(frames))
# wf.close()


# Listen for sinusoid that means "start of char-freq table"



## Receive Char-Freq Table

We'll need some already-known way to transfer this. 

## Decoding 

Listen for chunk, receive it, then decode and write to file **asynchronously** while listening to next chunk. 

### Listen for Chunk

### Decode Chunk

Use ECC. The goal is to do a one-pass over the file so we need to have *lots* of redundancy and capability to recover from errors. 

### Write to File

In [7]:
pls = np.sin(2 * np.pi * freq_range[0] * np.arange(t * samples) / samples)

In [8]:
np.shape(pls)

(10000,)

In [12]:
x = []

In [10]:
x = np.append(x, pls)

In [13]:
x.append(pls)

In [14]:
x

[array([ 0.00000000e+00,  1.22464680e-16, -2.44929360e-16, ...,
        -3.42467327e-13,  2.49173664e-12, -1.00302714e-12])]